In [55]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from numpy import *
from numpy.linalg import inv

In [23]:
class datasetgenerator:
    def __init__(self,size):
        self.train_arr = []
        self.size = size
        self.sigma = math.sqrt(0.1)
        self.Y = []
        self.gene()
        
    def gene(self):
        for _ in range(self.size):
            tmparr = []
            tmpfirst10 = random.normal(0,self.sigma,10)
            for var in tmpfirst10:
                tmparr.append(var)
            tmparr.append(tmparr[0]+tmparr[1]+np.random.normal(0,self.sigma))
            tmparr.append(tmparr[2]+tmparr[3]+np.random.normal(0,self.sigma))
            tmparr.append(tmparr[3]+tmparr[4]+np.random.normal(0,self.sigma))
            tmparr.append(0.1*tmparr[6]+np.random.normal(0,self.sigma))
            tmparr.append(2*tmparr[1]-10+np.random.normal(0,self.sigma))
            # x16~x20
            tmp16 = np.random.normal(0,self.sigma,5)
            for var in tmp16:
                tmparr.append(var)
            self.train_arr.append(tmparr)
            tmpy = self.computeY(tmparr)
            self.Y.append(tmpy)
            
    def computeY(self,arr):
        tmparr = arr
        y = 0
        i = 1
        for i in range(1,11):
            y+=(0.6)**i*tmparr[i-1]
        y+=10+np.random.normal(0,self.sigma)
        return y

In [151]:
d = datasetgenerator(1000)

In [152]:
class Linear_Regression:
    def __init__(self,train_arr,Y):
        self.arr_x = np.asarray(train_arr)
        self.arr_y = np.asarray(Y)
    
    def LSR(self):
        w = np.dot(inv(np.dot(self.arr_x.T,self.arr_x)),np.dot(self.arr_x.T,self.arr_y))
        return w
    
    def predict(self,w):
        err = 0
        arr_y = np.dot(self.arr_x,w)
        for i in range(len(self.arr_y)):
            err+=abs(arr_y[i]-self.arr_y[i])**2
        return err

In [153]:
def train(arr_x,arr_y):
    arr_x = np.asarray(arr_x)
    arr_y = np.asarray(arr_y)
    w = inv(np.dot(arr_x.T,arr_x))
    w2 = np.dot(arr_x.T,arr_y)
    w = np.dot(w,w2)
    return w

In [154]:
solver = Linear_Regression(d.train_arr,d.Y)

In [155]:
w = solver.LSR()

In [156]:
ans = solver.predict(w)

In [157]:
ans 

190.5105938667073